<a href="https://colab.research.google.com/github/Gustavo-br-rj/PROJ-BI_MASTER/blob/main/2_TESTA_PARAMETROS_RN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Previsão de Série Temporal da Temperatura de Mancais Utilizando LSTM


## Parte 0 - Importação das Bibliotecas

In [ ]:
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math


from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#tf.__version__

## Parte 1 - Importação do Dataset e Estatística Descritiva

In [ ]:
# Importação da Base de Dados
dataset = pd.read_csv('/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/DATASETS/dataset_C-4451_08001A.csv')

In [ ]:
dataset.groupby(['EQUIPAMENTO','MANCAL']).count()

DATA  TEMPERATURA
EQUIPAMENTO   MANCAL                                
C-4451.08001A EJA1.A-TC21.F_CV      132          132
              EJA1.A-TC22.F_CV      132          132
              EJA1.A-TC23.F_CV      132          132
              EJA1.A-TC25.F_CV      132          132
              EJA1.A-TC250.F_CV     133          133
              EJA1.A-TC251.F_CV     132          132
              EJA1.A-TC26.F_CV      132          132
              EJA1.A-TC27.F_CV      132          132
              EJA1.A-TC28.F_CV      132          132
              EJA1.A-TE1200A.F_CV   132          132
              EJA1.A-TE1200B.F_CV   132          132
              EJA1.A-TE1203A.F_CV   132          132
              EJA1.A-TE1203B.F_CV   132          132
              EJA1.A-TE1206A.F_CV   132          132
              EJA1.A-TE1206B.F_CV   132          132
              EJA1.A-TE1209A.F_CV   132          132
              EJA1.A-TE1209B.F_CV   132          132

In [ ]:
#Separação do dataset por mancais
mancais = dataset['MANCAL'].unique().tolist()

manc1 = dataset[dataset['MANCAL'] == mancais[0]]
manc2 = dataset[dataset['MANCAL'] == mancais[1]]
manc3 = dataset[dataset['MANCAL'] == mancais[2]]
manc4 = dataset[dataset['MANCAL'] == mancais[3]]
manc5 = dataset[dataset['MANCAL'] == mancais[4]]
manc6 = dataset[dataset['MANCAL'] == mancais[5]]
manc7 = dataset[dataset['MANCAL'] == mancais[6]]
manc8 = dataset[dataset['MANCAL'] == mancais[7]]
manc9 = dataset[dataset['MANCAL'] == mancais[8]]
manc10 = dataset[dataset['MANCAL'] == mancais[9]]
manc11 = dataset[dataset['MANCAL'] == mancais[10]]
manc12 = dataset[dataset['MANCAL'] == mancais[11]]
manc13 = dataset[dataset['MANCAL'] == mancais[12]]
manc14 = dataset[dataset['MANCAL'] == mancais[13]]
manc15 = dataset[dataset['MANCAL'] == mancais[14]]
manc16 = dataset[dataset['MANCAL'] == mancais[15]]
manc17 = dataset[dataset['MANCAL'] == mancais[16]]

## Parte 2 - Preprocessamento de Dados

### Separação do conjunto de Treino e Teste

In [ ]:
datasetMancal = manc15
#Conversão das temperaturas para número inteiro. Teste para ver se ajuda a RN. 
#datasetMancal = datasetMancal.astype({"TEMPERATURA": int})
#datasetMancal['TEMPERATURA'] = datasetMancal['TEMPERATURA'].round(decimals=1)

#Calculando tamanho dos conjuntos de treino e teste
train_size = math.ceil(len(datasetMancal)*0.85)
test_size  = len(datasetMancal) - train_size

#Separando conjuto de treino e teste apenas com valor da série
train_set = datasetMancal.iloc[0:train_size, 3:4].values
test_set  = datasetMancal.iloc[train_size:(train_size + test_size), 3:4].values

### Normalização

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train_set)

### Criar a estrutura de dados com janela WINDOW e output OUTPUTSIZE para entrada na RN

In [ ]:
def prepareSetRN(windowSize, outputSize, flatSet ):
  preparedX = []
  preparedY = []
  setSize = len(flatSet)
  for i in range(windowSize, setSize - (outputSize-1)):
    preparedX.append(flatSet[i-windowSize:i, 0])
    preparedY.append(flatSet[i:i + outputSize, 0])

  preparedX, preparedY = np.array(preparedX), np.array(preparedY)
  return preparedX, preparedY

## Parte 3 - Construção da RNN

##### Parâmetros para geração das redes

In [ ]:
windowSet   = [3,4,5,6]
outputSet   = [2,3]
layerSet    = [1,2,3]
unitSet     = [[150,80,80],[120,60,40],[80,80,60]]
dropoutSet  = [0.2, 0.25, 0.3, 0.35] 
optmizerSet = ['Adam', 'Adadelta']
epochs      = 800
batch_size  = 32

### Construção da Rede completa

In [ ]:
def criaRN(outputParam, layerParam, unitParam, dropoutParam, optmizerParam, X_trainParam):
  # Inicializar a RNN
  regressor = Sequential()

  # Adicionar a primeira camada LSTM e Dropout 
  regressor.add(LSTM(units = unitParam[0], return_sequences = (layerParam > 1), input_shape = (X_trainParam.shape[1], 1)))
  regressor.add(Dropout(dropoutParam))
 
  if ( layerParam >= 2 ):
    # Adicionar a segunda camada LSTM e Dropout
    regressor.add(LSTM(units = unitParam[1], return_sequences = (layerParam > 2)))
    regressor.add(Dropout(dropoutParam))

  if ( layerParam >= 3):
    # Adicionar a terceira camada LSTM e Dropout
    regressor.add(LSTM(units = unitParam[2], return_sequences = (layerParam > 3)))
    regressor.add(Dropout(dropoutParam))

  # camada de saída
  regressor.add(Dense(units = outputParam, activation='linear'))
  #Testar sigmoid e linear

  # Compilar a rede
  regressor.compile(optimizer = optmizerParam, loss = 'mean_squared_error', metrics=['accuracy'])

  # Visualizar a rede
  #regressor.summary()

  return regressor


In [ ]:
def backToFlat( windowSet ):
    flatSet     = []
    window_size = windowSet.shape[1]
    setSize     = len(windowSet)

    #Adiciona as primeiras colunas do primeiro registro quando a janela > 1
    if window_size > 1:
      for i in range(0, window_size-1):
        flatSet.append(windowSet[0, i])

    for i in range(0, setSize):
        flatSet.append(windowSet[i, window_size-1])

    flatSet = np.array(flatSet)

    return np.reshape(flatSet, (flatSet.shape[0], 1)) 

In [ ]:
diretorio = "/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/AVALIACAO_REDES/"
nomeArq   = '{0}__{1}_results.csv'.format(datasetMancal['EQUIPAMENTO'].to_numpy()[0], datasetMancal['MANCAL'].to_numpy()[0])
arquivo   = '{0}{1}'.format(diretorio,nomeArq)

fResults = open(arquivo,"w+")

fResults.write("EQUIPAMENTO,MANCAL,WINDOW,OUTPUT,NUM. CAMADAS,NUM. NEURONIOS CAMADA 1,NUM. NEURONIOS CAMADA 2,NUM. NEURONIOS CAMADA 3,DROPOUT,OTIMIZADOR,EPOCHS,BATCH_SIZE,RMSE,MSE,MAPE\r\n" )

for a in range(0, len(windowSet) ):
  for b in range(0, len(outputSet) ):
    for c in range(0, len(layerSet) ):
      for d in range(0, len(unitSet) ):
        for e in range(0, len(dropoutSet) ):
          for f in range(0, len(optmizerSet) ):
            X_train = []
            y_train = []
            X_train, y_train = prepareSetRN(windowSet[a], outputSet[b], training_set_scaled)
            X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

            regressor = criaRN(outputSet[b], layerSet[c], unitSet[d], dropoutSet[e], optmizerSet[f], X_train)
            history = regressor.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)
            
            #Acréscimo da janela no conjunto de teste para pegar os últimos registros do treino
            test_set_first_window  = dataset.iloc[train_size - windowSet[a]:(train_size + test_size), 3:4].values

            test_set_scaled = sc.transform(test_set_first_window)

            X_test = []
            y_test = []

            X_test, y_test = prepareSetRN(windowSet[a], outputSet[b], test_set_scaled)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

            predicted = regressor.predict(X_test)
            predicted = sc.inverse_transform(predicted)

            test_predicted = backToFlat(predicted)

            rmse = math.sqrt(mean_squared_error(test_set, test_predicted))
            mse = mean_squared_error(test_set, test_predicted)
            mape = np.mean(np.abs((test_set - test_predicted) / test_set)) * 100

            fResults.write("%s," % datasetMancal['EQUIPAMENTO'].to_numpy()[0])
            fResults.write("%s," % datasetMancal['MANCAL'].to_numpy()[0])
            fResults.write("%d," % windowSet[a])
            fResults.write("%d," % outputSet[b])
            fResults.write("%d," % layerSet[c])
            fResults.write("%d," % unitSet[d][0])
            fResults.write("%d," % unitSet[d][1])
            fResults.write("%d," % unitSet[d][2])
            fResults.write("%f," % dropoutSet[e])
            fResults.write("%s," % optmizerSet[f])
            fResults.write("%d," % epochs)
            fResults.write("%d," % batch_size)
            fResults.write("%f," % rmse)
            fResults.write("%f," % mse)
            fResults.write("%f" % mape)
            fResults.write("\r\n")

fResults.close()                    

Streaming output truncated to the last 5000 lines.
Epoch 768/800
4/4 [==============================] - 0s 9ms/step - loss: 0.0250 - accuracy: 0.6698
Epoch 769/800
4/4 [==============================] - 0s 11ms/step - loss: 0.0244 - accuracy: 0.6698
Epoch 770/800
4/4 [==============================] - 0s 8ms/step - loss: 0.0253 - accuracy: 0.6132
Epoch 771/800
4/4 [==============================] - 0s 8ms/step - loss: 0.0257 - accuracy: 0.6509
Epoch 772/800
4/4 [==============================] - 0s 9ms/step - loss: 0.0240 - accuracy: 0.6509
Epoch 773/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0258 - accuracy: 0.5943
Epoch 774/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0248 - accuracy: 0.6604
Epoch 775/800
4/4 [==============================] - 0s 8ms/step - loss: 0.0262 - accuracy: 0.6887
Epoch 776/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0265 - accuracy: 0.6509
Epoch 777/800
4/4 [==============================] - 0s 7